In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from langchain_openai import ChatOpenAI
import openai
from abrege.extractive_summary import EmbeddingModel
from abrege.summary_chain import new_summarize_chain

key = os.environ["OPENAI_API_KEY"]
base = os.environ["OPENAI_API_BASE"]
e_key = os.environ["OPENAI_EMBEDDING_API_KEY"]
e_base = os.environ["OPENAI_EMBEDDING_API_BASE"]

client = openai.OpenAI(
    base_url=e_base,
    api_key=e_key
)
em = EmbeddingModel(client)

llm = ChatOpenAI(
    base_url=base,
    api_key=key,
    streaming=True
)

chain = new_summarize_chain()

chain.invoke({
    "text": "J'adore le chocolat",
    "llm": llm,
    "embedding_model": em,
    "method": "map_reduce",
    "size": 100
})

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


'Le livre "J\'adore le chocolat" explore les thèmes principaux du chocolat. Il décrit les différentes façons dont l\'on peut apprécier ce délicieux dessert, en mettant en avant sa saveur sucrée et son arôme chaleureux. Le livre aborde également la passion qui peut être associée à l\'amour du chocolat, ainsi que les différentes façons dont cet amour peut se traduire dans la vie quotidienne. En fin de compte, le livre montre comment le chocolat peut être un élément clé d\'une relation amoureuse réussie et comment il peut renforcer l\'attrait mutuel entre les partenaires.'

In [28]:
from abrege.summary_chain import summarize_chain_builder
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callbacks = [StreamingStdOutCallbackHandler()]


chain = summarize_chain_builder(llm=llm, embedding_model=em, method="refine")
list(chain.stream({"text": "J'aime le chocolat"}, callbacks=callbacks))

TypeError: RunnableSequence._transform() got an unexpected keyword argument 'callbacks'

In [23]:
with open("../src/abrege/big_text.txt") as f:
    text = f.read()
async for s in chain.astream({"text": text}):
    print(s, end="\n", flush=True)

The| text| is| an| ex|cer|pt| from| the| novel| "|Tw|enty| Th|ous|and| Le|agues| Under| the| Sea|"| by| Jules| Ver|ne|.| It| describes| the| discovery| of| a| myster|ious| animal|,| which| the| narr|ator| and| his| compan|ions| observe| on| board| a| ship|.| The| creature| is| described| as| having| an| enorm|ous| body|,| with| a| thick| co|ating| of| electric|ity| around| it| that| makes| it| visible| in| the| dark|.| It| seems| to| be| able| to| move| quickly| through| the| water| and| even| catch|es| up| to| the| ship| at| one| point|.| The| narr|ator| and| his| compan|ions| are| unable| to| approach| or| attack| the| creature|,| as| they| do| not| know| its| nature| or| intent|ions|.| They| spec|ulate| about| what| kind| of| creature| it| might| be| and| whether| it| has| been| attacked| by| the| same| myster|ious| force| that| is| affect|ing| their| own| ship|.| The| text| ends| with| the| narr|ator| reflect|ing| on| the| strange| and| in|exp|lic|able| phen|omena| they| have| obse

In [34]:
url = "http://localhost:8000/stream_txt"
import requests

s = requests.Session()

with s.get(url, stream=True, params={"text": "J'ai très envie d'arrêter la programmation", "method": "map_reduce", "size": 300}) as r:
    for chunk in r.iter_content(1024):
        print(chunk)

b' The primary theme conveyed in the document "J\'ai tr\xc3\xa8s envie d\'arr\xc3\xaater la programmation" is an expressed desire to cease programming activities. This sentiment could stem from a multitude of underlying reasons, including but not limited to, seeking respite through a break, contemplating alternative career options, experiencing dissatisfaction with ongoing projects, or desiring to pursue interests beyond the realm of programming. The statement underscores an individual\'s inclination towards change and exploration outside their current professional domain. However, without further context from additional sources, it is difficult to ascertain more specific themes that may be associated with this desire to stop programming.'
